In [1]:
import numpy as np
import torch 
import pyro
import re
import pandas as pd

import seaborn as sns
import importlib
from plotnine import *
import time
import pickle

from granch_utils import init_model_tensor, main_sim_tensor, compute_prob_tensor, init_params_tensor, num_stab_help
#importlib.reload(granch_utils)
importlib.reload(num_stab_help)
importlib.reload(init_model_tensor)
importlib.reload(main_sim_tensor)

<module 'granch_utils.main_sim_tensor' from '/Users/caoanjie/Desktop/projects/looking_time_models/02_pyGRANCH/granch_utils/main_sim_tensor.py'>

In [2]:
importlib.reload(main_sim_tensor)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_INFO = {
    "jitter_n": 1, 
    "total_batch_n": 1, 
    "jitter_mode": "sampling"
}

GRID_INFO = {
    "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 20, 
    "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 20, 
    "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 20, 
    "grid_epsilon_start": 0.001, "grid_epsilon_end": 1.8, "grid_epsilon_step": 20, 
    "hypothetical_obs_grid_n": 10
}


BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

PRIOR_INFO = {
    "mu_prior": 0,  
    "V_prior": 3, 
    "alpha_prior": 1, 
    "beta_prior": 2, 
    "epsilon": 0.001, "mu_epsilon": 0.001, "sd_epsilon": 4, 
    "hypothetical_obs_grid_n": 10, 
    "world_EIGs": 0.0001, "max_observation": 500
}

tensor_stimuli = num_stab_help.sample_spore_experiment(1)

tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])


params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"])
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

res = main_sim_tensor.granch_main_simulation(params, tensor_model, tensor_stimuli[0])
print(res.behavior)
#main_sim_tensor.granch_main_simulation(PRIOR_INFO, tensor_model, tensor_stimuli)


{'total_batch_n': 1, 'jitter_n': 1, 'grid_mus': [tensor([-3.2303,  0.3490, -0.2922,  3.0763,  2.3303,  0.1790, -2.5570, -3.0810,
         0.2531, -0.9656, -0.6828,  3.4679, -1.5345, -2.6278,  2.6447,  0.4291,
        -2.6224,  0.9389,  2.7351, -0.9118])], 'grid_sigmas': [tensor([1.2718, 0.0663, 1.3698, 1.5920, 1.5505, 1.4414, 0.5247, 0.2465, 1.3000,
        0.5274, 0.8754, 0.4979, 1.4275, 0.3750, 1.3072, 0.0271, 1.4195, 0.3924,
        1.7746, 1.4666])], 'grid_ys': [tensor([ 1.0516, -3.9189, -0.2796,  1.6123, -3.8732, -1.4046, -2.8330,  0.4901,
         0.5528,  1.7594,  1.2130,  3.0808,  0.7425,  3.3384, -3.6554, -2.9351,
         2.1430,  3.9805,  0.2280, -2.6467])], 'grid_epsilons': [tensor([1.7119, 1.7640, 1.7517, 0.4069, 1.2638, 1.5843, 0.1613, 0.6149, 0.8493,
        0.7823, 1.1638, 0.3095, 1.6392, 1.1876, 1.2245, 1.7107, 0.4989, 0.3608,
        0.4227, 0.9957])]}
t 0


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:2895.)


t 1
t 2
t 3
t 4
t 5
t 6
t 7
t 8
t 9
t 10
t 11
t 12
t 13
t 14
t 15
t 16
t 17
t 18
t 19
t 20
t 21
t 22
t 23
t 24
t 25
    stimulus_id       EIG Look_away
0             0  1.532124     False
1             0  1.837647     False
2             0  1.181162     False
3             0  0.539963     False
4             0  0.228852     False
..          ...       ...       ...
495         NaN       NaN       NaN
496         NaN       NaN       NaN
497         NaN       NaN       NaN
498         NaN       NaN       NaN
499         NaN       NaN       NaN

[500 rows x 3 columns]
